In [3]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, coo_matrix, identity, vstack
from scipy.sparse.linalg import spsolve, lsqr
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline
# Tests of Openmesh
# https://gitlab.vci.rwth-aachen.de:9000/OpenMesh/openmesh-python/tree/master/tests


In [225]:

def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models):
    return trimesh.load(os.path.join('..', 'models', path_in_models))
def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)
def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)
def write_trimesh_with_color(mesh, path, colors):
    o3d_mesh = mesh.as_open3d
    o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    write_mesh_o3d(path, o3d_mesh)

In [5]:

### Calculate Voronoi Poles
INFINITY = 1e9
EPS = 1e-9
def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles


In [6]:

### Calculate Cotangent Laplace Operator
def trimesh_Generate_Area_List(mesh):

    area_list = np.zeros((len(mesh.vertices)))
    face_count = mesh.faces.shape[0]
    face_area = np.asarray(mesh.area_faces)
    faces = np.asarray(mesh.faces)
    for i in range(face_count):
        area_list[faces[i]] += face_area[i] / 3
    
    area_list /= np.mean(area_list)
    return area_list


def trimesh_Generate_Laplace_matrix(mesh):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(mesh)
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    print(len(faces))
    
    
    # with tqdm(total=face_count) as tbar:
    for face, angles in zip(faces, face_angles):
        for i in range(3):
            current_angle = angles[i]
            v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > EPS:
                laplace_dict[(v1_index, v1_index)] = laplace_dict.get((v1_index, v1_index), 0) - delta
                laplace_dict[(v2_index, v2_index)] = laplace_dict.get((v2_index, v2_index), 0) - delta
                laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + delta
                laplace_dict[(v2_index, v1_index)] = laplace_dict.get((v2_index, v1_index), 0) + delta
        # tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    print(delta,np.mean(values))
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    
    csr = coo.tocsr()

    return csr


In [7]:

### Solve Equation
def construct_Equation(wL, wH, wM, vertices, L, voronoi_poles):
    vn = vertices.shape[0]
    WH_L = wH * identity(vn)
    WM_L = wM * identity(vn)
    A = vstack((wL * L, WH_L, WM_L))
    b = np.vstack((np.zeros((vn, 3)), wH * vertices, wM * voronoi_poles))
    return A, b


In [8]:
### Laplace Smooth
flatten = lambda l: [item for sublist in l for item in sublist]

def laplacian_smoothing(mesh, smooth_factor=1.0, iterations=1):
   
    for _ in range(iterations):
        n = mesh.vertices.shape[0]
        vertex_degs = [len(neighbors) for neighbors in mesh.vertex_neighbors]
        sparse_mat_rows = [[row_idx] * (deg + 1) for row_idx, deg in enumerate(vertex_degs)]
        sparse_mat_cols = [neighbors + [row_idx] for row_idx, neighbors in enumerate(mesh.vertex_neighbors)]

        
        sparse_mat_weights = [[-1.0] * deg + [deg * 1.0] for deg in vertex_degs]   

        sparse_mat_rows = flatten(sparse_mat_rows)
        sparse_mat_cols = flatten(sparse_mat_cols) 

        sparse_mat_weights = [float(u) / row_weights[-1]
                                    for row_weights in sparse_mat_weights
                                    for u in row_weights]
        L =coo_matrix((sparse_mat_weights, (sparse_mat_rows, sparse_mat_cols)), shape=(n , n))
        L = L.tocsr()  # Compress the sparse matrix..
        v = np.array(mesh.vertices)
        diff = L * v
        mesh.vertices = v - smooth_factor * diff

In [9]:
def get_shared_face_index(mesh, v1, v2):
    # Get the faces containing vertex v1
    faces_with_v1 = np.where(np.any(mesh.faces == v1, axis=1))[0]
    # Get the faces containing vertex v2
    faces_with_v2 = np.where(np.any(mesh.faces == v2, axis=1))[0]
    # Find the intersection of the two arrays to get the shared face index
    shared_face_index = np.intersect1d(faces_with_v1, faces_with_v2)
    # print(shared_face_index)
    return shared_face_index

In [235]:
def get_adj_face(edges_unique, face_adjacency, v1, v2, face_index):
    if v1 > v2:
        v1, v2 = v2, v1
    e = np.where((edges_unique[:, 0]==v1)&(edges_unique[:, 1]==v2))
    faces = face_adjacency[e[0]].flatten()
    return faces[faces!=face_index].item()

# v1v0 onto v1v2
def cal_projection(v0, v1, v2):
    v1v2 = v2 - v1
    v1v0 = v0 - v1
    projector = v1v2 / np.linalg.norm(v1v2)
    projectee = v1v0
    t = np.dot(projectee, projector)
    return v1 + t * projector

def replace_vertex(face, v_old, v_new):
    new_face = face.copy()
    new_face[face==v_old] = v_new
    return new_face

In [159]:
face = np.array([1, 2, 4])
print(replace_vertex(face, 2, 5))

[1 5 4]


In [238]:
#         v0 ---------- v2
#        /        +     /
#      /      vn       /
#    /     +          /   
#  v1 ---------------v3
v_pairs = [[0,1],[1,2],[0,2]]
def reduce_Angles(mesh):

    faces = np.asarray(mesh.faces)
    edges_unique = np.asarray(mesh.edges_unique)
    face_adjacency = np.asarray(mesh.face_adjacency)
    face_angles = np.asarray(mesh.face_angles)
    vertices = np.asarray(mesh.vertices)

    f_is_del = np.zeros((len(mesh.faces)),dtype=bool) # f is deleted or not
    f_is_fixed = np.zeros((len(mesh.faces)),dtype=bool)
    f_to_add = np.asarray(mesh.faces) # all v indexs. Note that here it only adds points and dont remove any.
    v_to_add = np.asarray(mesh.vertices) # positions
    
    v_count = mesh.vertices.shape[0]
    face_count = faces.shape[0]
    
    for face_index in range(face_count):
        face = faces[face_index]
        if not f_is_fixed[face_index] and max(face_angles[face_index]) > (2./3.)*np.pi: # 2.1
            
            # get v0,v1,v2
            v0 = face[np.argmax(face_angles[face_index])] # obtuse_vertex
            v1, v2 = face[face!=v0] # v1,v2 are indexes.                

            # add vertex vn
            v_count += 1
            
            v_new = cal_projection(*vertices[[v0, v1, v2]])
            v_to_add = np.vstack((v_to_add, v_new))

            # get indexs of faces to be deleted
            face_adj_index = get_adj_face(edges_unique, face_adjacency, v1, v2, face_index)
            f_is_del[face_index]= True
            f_is_del[face_adj_index]= True
            face_adj = faces[face_adj_index]

            # fix adj faces in this iteration
            for i, j in v_pairs:
                f_is_fixed[get_adj_face(edges_unique, face_adjacency, 
                                        face[i], face[j], 
                                        face_index)] = True
                f_is_fixed[get_adj_face(edges_unique, face_adjacency, 
                                        face_adj[i], face_adj[j], 
                                        face_adj_index)] = True

            # # add faces
            vn = v_count - 1
            f_to_add = np.vstack((f_to_add, replace_vertex(face, v1, vn)))
            f_to_add = np.vstack((f_to_add, replace_vertex(face, v2, vn))) # the order matters! if go v0v2vn, not work
            f_to_add = np.vstack((f_to_add, replace_vertex(face_adj, v1, vn))) # why
            f_to_add = np.vstack((f_to_add, replace_vertex(face_adj, v2, vn)))                
            f_is_del = np.hstack((f_is_del, [False,False,False,False]))

    faces_new = f_to_add[~f_is_del]
    v_add = v_count - mesh.vertices.shape[0]

    print(mesh.face_adjacency.shape)
    print(mesh.is_watertight)

    mesh.vertices = v_to_add
    mesh.faces = faces_new

    print(mesh.face_adjacency.shape)
    print(mesh.is_watertight)

    return v_add

In [239]:

name = 'result/armadillo_0.obj'
mesh = read_mesh_trimesh(name)
# print(np.all(mesh.face_adjacency_span < np.pi))
# print(mesh.edges_unique)
print(mesh.faces.shape[0])
v_count = reduce_Angles(mesh)
print(v_count)
print(mesh.faces.shape[0])
print(mesh.outline())
# print(mesh.faces)
# print(mesh.faces) 
#print(v_count)
write_mesh_o3d('../models/result/test_cut_obtuse.obj', mesh.as_open3d)



50382
(75573, 2)
True
(82161, 2)
True
2196
54774
<trimesh.Path3D(vertices.shape=(27389, 3), len(entities)=0)>


In [218]:
def get_edge(mesh, v1, v2):
    edges_unique = np.asarray(mesh.edges_unique)
    if v1 > v2:
        v1, v2 = v2, v1
    e = np.where((edges_unique[:, 0]==v1)&(edges_unique[:, 1]==v2))
    return e[0].item()

faces = np.asarray(mesh.faces)
elist = []
for face_adj in tqdm(mesh.face_adjacency):
    ei = get_edge(mesh, *np.intersect1d(*faces[face_adj]))
    elist.append(ei)

100%|██████████| 84915/84915 [00:09<00:00, 8911.50it/s]


In [227]:
earray = np.array(elist)
eindices = np.arange(0, mesh.edges_unique.shape[0])
print(earray.shape, eindices.shape)
de = np.setdiff1d(eindices, earray)
v_color = np.ones_like(mesh.vertices)
c = np.array([1., 0., 0.])
for e in de:
    v1, v2 = mesh.edges_unique[e]
    v_color[v1] = c
    v_color[v2] = c
write_trimesh_with_color(mesh, '../models/result/test_cut_obtuse_color.obj', v_color)

(84915,) (85716,)


In [240]:
print(mesh.face_angles.max())
for i in range(5):
    v_count = reduce_Angles(mesh)
    print(v_count)
print(mesh.face_angles.max())# ? biandale  wcao

3.1392468069237287
(82161, 2)
True
(88104, 2)
True
1981
(88104, 2)
True
(93390, 2)
True
1762
(93390, 2)
True
(97677, 2)
True
1429
(97677, 2)
True
(101391, 2)
True
1238
(101391, 2)
True
(104517, 2)
True
1042
3.141445919043002


In [229]:
# Introduction and Criticism

# to make sure obtuse is smaller, the threshold make be bigger than 120 degree
# x/2 + 180 - x < x ==> x > 120

#     A
#    /
#   /
#  B-------------D-------------C
# BAC is X, if ACD exist and ADC < BAC(obtuse smaller), then x/2+180-x<x

# 110 will not gurantee this!! (110 + 69 + 1） will make a bigger obtuse!

In [47]:
### Iteration
it = 1
wL_list = [20]#[20,10,1.5,1,0.5,0.05,0.005,0.0005,0.00001]
wH_list = [0.5]#[20,10,1.5,1,0.5,0.05,0.005,0.0005,0.00001]
wM_list = [1.5]#[20,10,1.5,1,0.5,0.05,0.005,0.0005,0.00001]
name = 'armadillo'
new_mesh = name + '_0_smooth.obj'
# new_mesh = 'asculpt.obj'
mesh = read_mesh_trimesh(name+'.obj')
voronoi_poles = calculate_voronoi_poles(mesh)
for wL in wL_list:
    for wH in wH_list:
        for wM in wM_list:
            for i in range(it):
                mesh = read_mesh_trimesh(name+'.obj')
                print("processing"+str(wL)+str(wH)+str(wM))

                laplace_csr = trimesh_Generate_Laplace_matrix(mesh)
                # if i > 0:
                #     laplace_csr *= -10 * np.power(10,i)
                # print(np.mean(laplace_csr))
                vertices = get_vertices_from_trimesh(mesh)
                A, b = construct_Equation(wL, wH, wM, vertices, laplace_csr, voronoi_poles)

                new_vertices0 = lsqr(A, b[:, 0])
                new_vertices1 = lsqr(A, b[:, 1])
                new_vertices2 = lsqr(A, b[:, 2])

                new_vertices = np.zeros(vertices.shape)
                new_vertices[:, 0] = new_vertices0[0]
                new_vertices[:, 1] = new_vertices1[0]
                new_vertices[:, 2] = new_vertices2[0]

                mesh.vertices = new_vertices
                
                # save_index = int(i/3)
                # mesh = mesh.smoothed()
                # mesh = mesh.subdivide_to_size(average_edge_length(mesh))
                # laplacian_smoothing(mesh,smooth_factor = 1.0)
                # mesh = subdivide_based_on_angle(mesh, 0.9)
                new_mesh = name +str(i)+'_%d_smooth.obj' 
                write_mesh_o3d('../models/result/' + new_mesh, mesh.as_open3d)
                # mesh = read_mesh_trimesh('result/' + new_mesh)
                # laplacian_smoothing(mesh,smooth_factor = 1.0)
                # new_mesh = name +str(wL)+str(wH)+str(wM) +'_%d_smooth.obj' % (save_index + 1)
                # smoothed_mesh = '%d_smooth.obj' % (save_index + 1)
                # write_mesh_o3d('../models/result/' + smoothed_mesh, mesh.as_open3d)
                # print("Done one time-----------------------------")
            
(mesh.face_angles).max()


Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:01<00:00, 20742.52it/s]


processing200.51.5
Constructing Laplace Matrix.
50382
2.482933512073437 0.0


3.130051856485212

In [ ]:
# if threshold: 1.8 3.1300 -> 3.1353
# if 2.1 : 

In [ ]:
def collapse_mesh(mesh):
    vertices = mesh.vertices
    faces = mesh.faces